In [6]:


from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent))
import networkx as nx
import numpy as np
from matrix import are_similar
from itertools import combinations
from numpy import trace
from networkx import is_bipartite




tolerance = 1e-10


In [7]:
def get_least_cell_quotient(adj_matrix):
    """
    Compute the quotient matrix for the coarsest equitable partition of a graph.
    
    An equitable partition is a partition where the number of edges from a node
    to cells depends only on which cell the node is in, not the specific node.
    
    Parameters:
    -----------
    adj_matrix : array-like
        The adjacency matrix of the graph
    
    Returns:
    --------
    quotient_matrix : np.ndarray
        The quotient matrix (each entry is the number of edges between cells)
    partition : np.ndarray
        The partition of vertices (cell assignment for each vertex)
    """
    adj = np.array(adj_matrix)
    n = adj.shape[0]
    
    # 1. Start with all nodes in the same partition (color 0)
    partition = np.zeros(n, dtype=int)
    
    while True:
        # 2. For each node, create a signature based on its neighbors' partitions
        # Signature = (current_partition, sorted_list_of_neighbor_partitions)
        signatures = []
        for i in range(n):
            neighbor_parts = sorted(partition[j] for j in range(n) if adj[i, j] > 0)
            signatures.append((partition[i], tuple(neighbor_parts)))
        
        # 3. Map unique signatures to new partition labels (colors)
        unique_sigs = sorted(list(set(signatures)))
        sig_map = {sig: idx for idx, sig in enumerate(unique_sigs)}
        new_partition = np.array([sig_map[sig] for sig in signatures])
        
        # 4. If the partition hasn't changed, we've found the coarsest equitable partition
        if np.array_equal(new_partition, partition):
            break
        partition = new_partition

    # 5. Build the quotient matrix
    num_cells = len(unique_sigs)
    quotient_matrix = np.zeros((num_cells, num_cells))
    
    cells = [np.where(partition == i)[0] for i in range(num_cells)]
    
    for i in range(num_cells):
        for j in range(num_cells):
            # Pick any representative node 'u' from cell i
            u = cells[i][0]
            # Count edges from 'u' to all nodes in cell j
            edge_count = sum(adj[u, v] for v in cells[j])
            quotient_matrix[i, j] = edge_count
            
    return quotient_matrix, partition

In [8]:


n = 6

path = f"../graphs/graph{n}c.g6.txt"
f = open(path, "r")

candidates = []

for line in f:
    g6 = line.strip()
    G = nx.from_graph6_bytes(g6.encode())
    A = nx.to_numpy_array(G, dtype=int)
    # One representative per similarity class, per loop count (here 1 loop)
    
    for i in range(1, n):
        for comb in combinations(range(n), i):
            B = A.copy()
            for j in comb:
                B[j, j] = 1
            eigenvalues = np.linalg.eigvalsh(B)
            rounded_eigenvalues = np.round(eigenvalues / tolerance) * tolerance
            unique_eigenvalues = np.unique(rounded_eigenvalues)


            if len(unique_eigenvalues) == 3:
                is_duplicate = any(trace(B) == trace(C) and are_similar(B, C) for C in candidates)
                if not is_duplicate and is_bipartite(G):
                    D = A.copy()
                    for j in range(n):
                        if j not in comb:
                            D[j, j] = 1
                    is_duplicate = any(trace(D) == trace(C) and are_similar(D, C) for C in candidates)
                if not is_duplicate:
                    candidates.append(B)
                    print(B)
                    print(unique_eigenvalues)
                    print("\n")
    

[[0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [1 1 1 1 1 1]]
[-1.79128785 -0.          2.79128785]


[[0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
[-2. -0.  4.]


[[0 0 0 1 1 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 1]
 [1 1 0 1 1 1]
 [1 0 1 1 1 1]
 [0 1 1 1 1 1]]
[-1.  1.  4.]


[[0 0 0 1 1 1]
 [0 0 0 1 1 1]
 [0 0 0 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
[-1.85410197 -0.          4.85410197]


[[0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [1 0 0 0 1 1]
 [0 1 0 0 0 1]
 [1 0 1 0 0 1]
 [1 1 1 1 1 1]]
[-1.          1.38196601  3.61803399]


[[0 0 1 1 1 1]
 [0 0 0 0 0 1]
 [1 0 0 1 1 1]
 [1 0 1 0 1 1]
 [1 0 1 1 0 1]
 [1 1 1 1 1 1]]
[-1.          0.69722436  4.30277564]


[[1 0 1 1 1 1]
 [0 1 1 1 1 1]
 [1 1 1 0 1 1]
 [1 1 0 1 1 1]
 [1 1 1 1 0 1]
 [1 1 1 1 1 0]]
[-1.  1.  5.]


[[1 0 1 1 1 1]
 [0 1 1 1 1 1]
 [1 1 0 1 1 1]
 [1 1 1 0 1 1]
 [1 1 1 1 0 1]
 [1 1 1 1 1 0]]
[-1.  1.  5.]


[[0 0 1 1 1 1]
 [0 0 1 1 1 1]
 [1 1 1 1 1 1]
 [1

In [9]:
l_shape = []
row_sum_candidates = []
l_shape_row_sum_candidates = []
others = []

for candidate in candidates:
    row_sum = np.sum(candidate, axis=1)
    unique_row_sum = np.unique(row_sum)
    x1, x2 = unique_row_sum if len(unique_row_sum) == 2 else (0, 0)
    if all(row_sum == row_sum[0]):
        row_sum_candidates.append(candidate)
    elif len(unique_row_sum) == 2 and (x1 == n and sum(row_sum == n) == x2 or x2 == n and sum(row_sum == n) == x1):
        l_shape.append(candidate)
    else:
        sub_matrix = candidate.copy()
        mask = row_sum != n
        sub_matrix = candidate[np.ix_(mask, mask)]
        sub_matrix_row_sum = np.sum(sub_matrix, axis=1)
        mask = sub_matrix_row_sum != 0
        if all(sub_matrix_row_sum[mask] == sub_matrix_row_sum[mask][0]):
            l_shape_row_sum_candidates.append(candidate)
        else:
            others.append(candidate)


In [10]:
print("num of candidates: ", len(candidates))
print("num of row sum candidates: ", len(row_sum_candidates))
print("num of l shape candidates: ", len(l_shape))
print("num of l shape row sum candidates: ", len(l_shape_row_sum_candidates))
print("num of others: ", len(others))




num of candidates:  11
num of row sum candidates:  2
num of l shape candidates:  5
num of l shape row sum candidates:  2
num of others:  2


In [11]:
for other in others:
    print(other)
    print(np.sum(other, axis=1))
    print("\n")

[[0 0 0 1 1 0]
 [0 0 0 1 0 1]
 [0 0 0 0 1 1]
 [1 1 0 1 1 1]
 [1 0 1 1 1 1]
 [0 1 1 1 1 1]]
[2 2 2 5 5 5]


[[0 0 1 0 1 1]
 [0 0 0 1 0 1]
 [1 0 0 0 1 1]
 [0 1 0 0 0 1]
 [1 0 1 0 0 1]
 [1 1 1 1 1 1]]
[3 2 3 2 3 6]




In [12]:
# Example: Compute quotient matrices for candidates
# This creates the best/coarsest equitable partition for each adjacency matrix

for idx, candidate in enumerate(candidates[:5]):  # Show first 5 examples
    q_mat, part = get_least_cell_quotient(candidate)
    print(f"Candidate {idx}:")
    print("Quotient Matrix:\n", q_mat.astype(int))
    print("Vertex Partitions:", part)
    print()


Candidate 0:
Quotient Matrix:
 [[0 1]
 [5 1]]
Vertex Partitions: [0 0 0 0 0 1]

Candidate 1:
Quotient Matrix:
 [[0 2]
 [4 2]]
Vertex Partitions: [0 0 0 0 1 1]

Candidate 2:
Quotient Matrix:
 [[0 2]
 [2 3]]
Vertex Partitions: [0 0 0 1 1 1]

Candidate 3:
Quotient Matrix:
 [[0 3]
 [3 3]]
Vertex Partitions: [0 0 0 1 1 1]

Candidate 4:
Quotient Matrix:
 [[1 0 1]
 [0 2 1]
 [2 3 1]]
Vertex Partitions: [1 0 1 0 1 2]

